<a href="https://colab.research.google.com/github/JesusRiveroOrtega/heart_sounds_segmentation_ConvLSTM/blob/master/hearbeat_sounds_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.io import wavfile as wf
import numpy as np
import random
from matplotlib import pyplot as plt
import csv
from scipy import signal
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:
!cd 'drive/My Drive/kaggle/heartbeat_sounds' && ls

heartbeat-sounds.zip  set_a.csv		set_a_timing.gsheet  set_b.csv
set_a		      set_a_timing.csv	set_b


In [0]:
a_name = ''
p_name = ''
files = 0
target = [[[],[]]]
k = 0
AUDIO = np.zeros((21, 396900))
TARGET = np.zeros((3, 21, 396900))
#Data loading process
#The dataset is composed by several audio files, the file set_a_timing.csv contains the name of the audio files
#The following process takes the name in the .csv file and load the audio file to append it to a list with the rest of audios

with open('drive/My Drive/kaggle/heartbeat_sounds/set_a_timing.csv') as csv_file:  
    csv_reader = csv.reader(csv_file, delimiter=',') #Function to read .csv files
    line_count = 0    
    for row in csv_reader: #this for loop takes each row of the .csv file
      if line_count > 0:
        a_name = row[0].split('/')[1] #The first column of the .csv file contains the names, to
        if a_name == p_name:
          target[k][0].append(row[2])
          target[k][1].append(row[3])
          if row[2] == 'S1':
            TARGET[0, k-1, int(row[3])] = 1
          if row[2] == 'S2':
            TARGET[1, k-1, int(row[3])] = 1
        else:
          target.append([[row[2]],[row[3]]])
          
          fs, data = wf.read('drive/My Drive/kaggle/heartbeat_sounds/set_a/'+a_name)   
          fc = 410
          
          b, a = signal.butter(4, fc/(fs/2), 'low', analog=True)
          data = signal.filtfilt(b, a, data)
          data = signal.filtfilt(b, a, data)
          mean = np.mean(data)
          data = data-mean
          std = np.std(data)
          data/=std
          if np.shape(data)[0] < 396900:  
            
            tmp_data = np.pad(data, (0,int(396900 - np.shape(data)[0])), 'constant', constant_values=(0, 0))
            
            AUDIO[k, :] = np.interp(tmp_data, (tmp_data.min(), tmp_data.max()), (-1, +1))           
          else:      
            
            AUDIO[k, :] = np.interp(data, (data.min(), data.max()), (-1, +1))
          
          k += 1
        
      
      else:
        pass
      p_name = a_name      
      line_count+=1
AUDIO = np.interp(AUDIO, (AUDIO.min(), AUDIO.max()), (-1, +1))
#AUDIO_der = np.gradient(AUDIO, axis=1)
#AUDIO = np.concatenate((AUDIO, AUDIO))
target = target[1:]


In [0]:
AUDIO_r = np.ravel(AUDIO)
DATA_S1 = []
DATA_S2 = []
DATA_ZE = []
TARG_S1 = []
TARG_S2 = []
TARG_ZE = []
TARGET_r = np.reshape(TARGET, (3, -1))
tg = TARGET_r[0,:]+TARGET_r[1,:]
win = 0
width = 4000
overlap = 500
while win < AUDIO_r.shape[0] - width:
  
  if (1 in TARGET_r[0, win:win+width]) and (0 in TARGET_r[1, win:win+width]):
    TARG_S1.append(np.array([1, 0, 0]))
    DATA_S1.append(AUDIO_r[win:win+width])
  if (0 in TARGET_r[0, win:win+width]) and (1 in TARGET_r[1, win:win+width]):
    TARG_S2.append(np.array([0, 1, 0]))
    DATA_S2.append(AUDIO_r[win:win+width])
  else:
    TARG_ZE.append(np.array([0, 0, 1]))
    DATA_ZE.append(AUDIO_r[win:win+width])
  win += overlap
  

In [0]:

S1 = np.hstack((np.array(DATA_S1), np.array(TARG_S1)))
S2 = np.hstack((np.array(DATA_S2), np.array(TARG_S2)))
ZE = np.hstack((np.array(DATA_ZE), np.array(TARG_ZE)))
for i in range(0, 20):
  S1 = S1[np.random.permutation(range(0, S1.shape[0])), :]
  S2 = S2[np.random.permutation(range(0, S2.shape[0])), :]
  ZE = ZE[np.random.permutation(range(0, ZE.shape[0])), :]

s1 = S1.shape[0]
s2 = S2.shape[0]
ze = ZE.shape[0]

if s1 > s2:
  S1_DAT = S1[0:s2,:]
  S2_DAT = S2
  ZE_DAT = ZE[0:s2,:]
if s2 > s1:
  S2_DAT = S2[0:s1,:]
  S1_DAT = S1
  ZE_DAT = ZE[0:s1,:]
  
TRAINING_SET_PREV = np.vstack((S1_DAT[0:int(S1_DAT.shape[0] * 0.8), :], S2_DAT[0:int(S2_DAT.shape[0] * 0.8), :], ZE_DAT[0:int(ZE_DAT.shape[0] * 0.8), :]))  
TESTING_SET_PREV = np.vstack((S1_DAT[int(S1_DAT.shape[0] * 0.8):, :], S2_DAT[int(S2_DAT.shape[0] * 0.8):, :], ZE_DAT[int(ZE_DAT.shape[0] * 0.8):, :]))


TRAINING_DATA = TRAINING_SET_PREV[:, 0:TRAINING_SET_PREV.shape[1] - 3]
TRAINING_TARG = TRAINING_SET_PREV[:, TRAINING_SET_PREV.shape[1] - 3:]

TESTING_DATA = TRAINING_SET_PREV[:, 0:TRAINING_SET_PREV.shape[1] - 3]
TESTING_TARG = TRAINING_SET_PREV[:, TRAINING_SET_PREV.shape[1] - 3:]

#for i in range(0, 20):
  #TESTING_DATA = TESTING_DATA[np.random.permutation(range(0, TESTING_DATA.shape[0])), :]
  #TESTING_TARG = TESTING_TARG[np.random.permutation(range(0, TESTING_TARG.shape[0])), :]

In [0]:
print(TRAINING_DATA.shape)
TRAINING_DATA = np.expand_dims(TRAINING_DATA, axis=2)
TESTING_DATA = np.expand_dims(TESTING_DATA, axis=2)
TRAINING_DATA = np.concatenate((TRAINING_DATA, np.gradient(TRAINING_DATA, axis=1)), axis=2)
TESTING_DATA = np.concatenate((TESTING_DATA, np.gradient(TESTING_DATA, axis=1)), axis=2)

(3339, 4000)


In [0]:
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
import tensorflow as tf

Using TensorFlow backend.


In [0]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 400, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], 2, trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 50, 64
  trainX = trainX.reshape((trainX.shape[0], n_steps, 1, -1, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, 1, -1, n_features))
  print(testX.shape[1:])
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=32, kernel_size=(1,3), activation='relu', input_shape = testX.shape[1:], return_sequences = True)) 
  model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', return_sequences = True))
  model.add(ConvLSTM2D(filters=128, kernel_size=(1,3), activation='relu', return_sequences = False))
  model.add(Dropout(0.5))
  model.add(Flatten())  
  model.add(Dense(300, activation='relu'))     
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy, model
 
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=2):	
	# repeat experiment
  scores = list()
  for r in range(repeats):
    score, model = evaluate_model(TRAINING_DATA, TRAINING_TARG, TESTING_DATA, TESTING_TARG)
    name = '_HBS_V2_2_acc_'
    model.save_weights('drive/My Drive/weights'+ name + str(score*100) +'.h5')
    with open('drive/My Drive/model' + name + str(score*100) +'.json', 'w') as f:
      f.write(model.to_json())

    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
    # summarize results
    summarize_results(scores)
  return model

In [0]:
# run the experiment
model = run_experiment()


W0807 19:37:17.325997 139737227839360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 19:37:17.365006 139737227839360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 19:37:17.372385 139737227839360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



(50, 1, 80, 2)


W0807 19:37:18.406865 139737227839360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0807 19:37:18.418760 139737227839360 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0807 19:37:18.472294 139737227839360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0807 19:37:18.512003 139737227839360 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tenso

In [0]:
  !ls && cd drive && ls

In [0]:
#from keras.models import model_from_json

# Model reconstruction from JSON file
#with open('drive/My Drive/model_HBS_V2_1_acc_95.87523946360153.json', 'r') as f:
    #model = model_from_json(f.read())

# Load weights into the new model
#model.load_weights('drive/My Drive/weights_HBS_V2_1_acc_95.87523946360153.h5')

In [0]:
audio_example = AUDIO[0, :]
prediction = []
counter = 0
overlap = 500
print(width)
while True:
  if counter+width > audio_example.shape[0]:
    audio_example_piece = audio_example[counter:counter+width]
    audio_example_piece = np.hstack((audio_example_piece, np.zeros((width - audio_example_piece.shape[0]))))
    audio_example_piece = np.expand_dims(audio_example_piece, axis=1)
    audio_example_piece = np.expand_dims(audio_example_piece, axis=2)  
    audio_example_piece = np.concatenate((audio_example_piece, np.gradient(audio_example_piece, axis = 0)), axis=2)  
    audio_example_piece = audio_example_piece.reshape((1, 20, 1, -1, 2))
    prediction.append(model.predict(audio_example_piece))
    break
  else:
    audio_example_piece = audio_example[counter:counter+width]
    audio_example_piece = np.expand_dims(audio_example_piece, axis=1)
    audio_example_piece = np.expand_dims(audio_example_piece, axis=2)  
    audio_example_piece = np.concatenate((audio_example_piece, np.gradient(audio_example_piece, axis = 0)), axis=2)  
    audio_example_piece = audio_example_piece.reshape((1, 20, 1, -1, 2))
    prediction.append(model.predict(audio_example_piece))
  
  counter += overlap
print()

In [0]:
print(prediction[0])

In [0]:
predicted_targ = np.zeros((audio_example.shape[0], 2))
counter = 0

for pred in prediction:
  pred = pred[0]
  if overlap == 4000:
    if np.argmax(pred) == 0 and pred[np.argmax(pred)] > 0.90:
      predicted_targ[counter:counter+width, 0] = np.ones((width))
    if np.argmax(pred) == 1 and pred[np.argmax(pred)] > 0.96:
      predicted_targ[counter:counter+width, 1] = np.ones((width))
  else:
    if np.argmax(pred) == 0 and pred[np.argmax(pred)] > 0.90:
      predicted_targ[counter:counter+width, 0] = np.ones((predicted_targ[counter:counter+width, 0].shape[0]))
    if np.argmax(pred) == 1 and pred[np.argmax(pred)] > 0.96:
      predicted_targ[counter:counter+width, 1] = np.ones((predicted_targ[counter:counter+width, 1].shape[0]))
  counter += overlap

In [0]:
for i in range(0, 1):
  plt.figure(num=1, figsize=(50, 6), dpi=90, facecolor='w', edgecolor='k')
  plt.plot(audio_example, 'b')
  plt.plot(predicted_targ[:,0], 'k')
  plt.plot(predicted_targ[:,1], 'c')
  for k in range(0, len(target[i][0])):
    if target[i][0][k] == 'S1':
      plt.axvline(float(target[i][1][k]), color = 'k')
    else:
      plt.axvline(float(target[i][1][k]), color = 'c')
    plt.text(float(target[i][1][k]),0,target[i][0][k],rotation=90, fontsize = 12, color='r')
  plt.show()

In [0]:
for i in range(0, 1):
  plt.figure(num=1, figsize=(50, 6), dpi=90, facecolor='w', edgecolor='k')
  plt.plot(AUDIO[i, :], 'b')
  plt.plot(TA)
  for k in range(0, len(target[i][0])):
    if target[i][0][k] == 'S1':
      plt.axvline(float(target[i][1][k]), color = 'r')
    else:
      plt.axvline(float(target[i][1][k]), color = 'g')
    plt.text(float(target[i][1][k]),0,target[i][0][k],rotation=90, fontsize = 12)
  plt.show()

In [0]:
PRED = np.array(prediction)

plt.plot(AUDIO[0,:], 'b')
plt.plot(PRED[0,:], 'c')
plt.plot(PRED[1,:], 'm')
plt.show()